In [ ]:
from flask import Flask, request, jsonify
import string
import speech_recognition as sr
import subprocess
import os

app = Flask(__name__)

@app.route('/upload-audio', methods=['POST'])
def upload_audio():
    audio_file = request.files.get('audio')
    if audio_file is None:
        return jsonify({'error': 'No audio file provided'})

    try:
        audio_file.save('audio-file.wav')
        print("Audio file saved")
    except Exception as e:
        return jsonify({'error': 'Failed to save audio file'})
    try:
        os.remove('output_file.wav')
    except Exception as e:
        pass
    try:
        subprocess.call(['ffmpeg', '-i', 'audio-file.wav', 'output_file.wav'])
    except Exception as e:
        return jsonify({'error': 'Failed to convert audio file'})

    room = ""
    switch = ""
    value = ""
    r = sr.Recognizer()
    with sr.AudioFile('output_file.wav') as source:
        try:
            audio = r.record(source)
            text = r.recognize_google(audio, language='ur-PK')
            print("You said:", text)
            words = text.split()
            print("Words:", words)

            if "پہلے" in words:
                room = "room1"
                print("پہلے")
            elif "دوسرے" in words:
                room = "room2"
                print("دوسرے")
            elif "تیسرے" in words:
                room = "room3"
                print("تیسرے")

            if "بلب" in words:
                switch = "Light 1"
                print("بلب")
            elif "بتی" in words:
                switch = "Light 2"
                print("بتی")
            elif "پنکھا" in words:
                switch = "Fan"
                print("پنکھا")

            if "چلا" in words:
                value = "ON"
                print("چلاؤ")
            elif "بند" in words:
                value = "OFF"
                print("بند")
        except sr.UnknownValueError:
            print("Could not understand audio")
            return jsonify({'error': 'Could not understand audio'})
        except sr.RequestError as e:
            print("Could not request results; {0}".format(e))
            return jsonify({'error': 'Failed to process audio'})

    # Remove the output file after processing
    try:
        os.remove('output_file.wav')
    except Exception as e:
        pass

    return jsonify({'Room': room, "Switch": switch, "Value": value,"Text":text})


if __name__ == '__main__':
    app.run(host='192.168.21.197', port=5000)
